# Overview

This notebook provides a top down view of global equity markets across all sectors and styles based on the cyclically-adjusted price/earnings ratio, or CAPE. 

## What is the CAPE ratio?

In 1998, Robert Shiller and John Campbell published the pathbreaking article “Valuation Ratios and the Long-Run Stock Market Outlook.” A follow-up to some of their earlier work on stock market predictability, it established that long-term stock market returns were not random walks but, rather, could be forecast by a valuation measure called the “cyclically adjusted price–earnings ratio,” or CAPE ratio. Shiller and Campbell calculated the CAPE ratio by dividing a long-term broad-based index of stock market prices and earnings from 1871 by the average of the last 10 years of earnings per share, with earnings and stock prices measured in real terms. They regressed 10-year real stock returns against the CAPE ratio and found that the CAPE ratio is a significant variable that can predict long-run stock returns. The predictability of real stock returns implies that long-term equity returns are mean reverting. In other words, if the CAPE ratio is above (below) its long-run average, the model predicts belowaverage (above-average) real stock returns for the next 10 years. 

*Jeremy J. Siegel (2016) The Shiller CAPE Ratio: A New Look, Financial Analysts Journal, 72:3, 41-50, DOI: 10.2469/faj.v72.n3.1*

In [39]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [7]:
classifications = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/classification_data.csv', index_col=0).iloc[:, :-2]
grinold_kroner = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/grinold_kroner_returns.csv', index_col=0)
current_fwd_return_5y_forecast = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/current_fwd_return_5y_forecast.csv', index_col=0)
benchmark_prices = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/etp_prices.csv', index_col=0)
benchmark_lt_pe = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/benchmark_lt_pe.csv', index_col=0)

classifications.index.name = 'BENCHMARK_TICKER'
grinold_kroner.index.name = 'BENCHMARK_TICKER'
current_fwd_return_5y_forecast.index.name = 'BENCHMARK_TICKER'

In [25]:
results = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/equity_etf_posterior_returns.csv')
results.columns = ['BENCHMARK_TICKER', 'ETF_TICKER', 'CORRELATION', 'P_VALUE', 'BENCHMARK_NAME', 'PRIOR_RETURN', 'POSTERIOR_RETURN', 'VIEW']
results = results[['ETF_TICKER', 'CORRELATION', 'P_VALUE', 'PRIOR_RETURN', 'POSTERIOR_RETURN', 'VIEW', 'BENCHMARK_NAME', 'BENCHMARK_TICKER']]
results.ETF_TICKER = results.ETF_TICKER.str.replace(' US Equity', '')
results['ETF_NAME'] = classifications.loc[results.ETF_TICKER.values].NAME.values
results['CLASSIFICATION'] = classifications.loc[results.ETF_TICKER.values].CLASSIFICATION.values
results = results.set_index('ETF_TICKER')
results = pd.merge(results, grinold_kroner, left_on='BENCHMARK_TICKER', right_index=True, how='left')
results = pd.merge(results, current_fwd_return_5y_forecast.FWD_RETURN_5Y_FORECAST, left_on='BENCHMARK_TICKER', right_index=True, how='left')
results = results[results['CORRELATION']>=0.95].sort_values(by='POSTERIOR_RETURN', ascending=False)
results.head()

,CORRELATION,P_VALUE,PRIOR_RETURN,POSTERIOR_RETURN,VIEW,BENCHMARK_NAME,BENCHMARK_TICKER,ETF_NAME,CLASSIFICATION,LONG_TERM_EARNINGS_YIELD,NOMINAL_EARNINGS_GROWTH,REPRICING_RETURN,GRINOLD_KRONER_RETURN,FWD_RETURN_5Y_FORECAST
ETF_TICKER,,,,,,,,,,,,,,
PSCD,0.9993,0,0.184952,0.121506,0.17615,S&P 600 Consumer Discretionary Sector GICS Lev...,S6COND Index,Invesco S&P SmallCap Consumer Discretionary ETF,U.S. Small-cap Value ETP,0.0781,0.0375,0.0178,0.1334,0.2189
XRT,0.9682,0,0.184952,0.121506,0.17615,S&P 600 Consumer Discretionary Sector GICS Lev...,S6COND Index,SPDR S&P Retail ETF,U.S. Broad Market Blend ETP,0.0781,0.0375,0.0178,0.1334,0.2189
VIOV,0.9924,0,0.164868,0.096484,0.13455,S&P Small Cap 600 Value Index,SMLV Index,Vanguard S&P Small-Cap 600 Value ETF,U.S. Small-cap Value ETP,0.0683,0.0375,0.0040,0.1098,0.1593
FYT,0.9634,0,0.164868,0.096484,0.13455,S&P Small Cap 600 Value Index,SMLV Index,First Trust Small Cap Value AlphaDEX Fund,U.S. Small-cap Value ETP,0.0683,0.0375,0.0040,0.1098,0.1593
IJS,0.9933,0,0.164868,0.096484,0.13455,S&P Small Cap 600 Value Index,SMLV Index,iShares S&P Small-Cap 600 Value ETF,U.S. Small-cap Value ETP,0.0683,0.0375,0.0040,0.1098,0.1593


In [37]:
periodicity_table = {
    'DAILY': 252,
    'WEEKLY': 52,
    'MONTHLY': 12,
    'QUARTERLY': 4,
    'YEARLY': 1
}

In [38]:
periodicity = 'MONTHLY'

In [ ]:
sample_fwd_return_5y = ((etp_prices.shift(-periodicity_table.get(periodicity)*5)/etp_prices)**(1/5)-1).round(4).dropna()

In [ ]:
sample_lt_pe = np.log(lt_pe.loc[sample_fwd_return_5y.index])

In [ ]:
results_dict = {}
for ticker in tqdm(list(lt_pe.columns)):
    y = sample_fwd_return_5y[ticker]
    x = sample_lt_pe[ticker]
    X = sm.add_constant(x)
    results_dict[ticker] = sm.OLS(y, X).fit().get_robustcov_results('HAC', maxlags=1) # 'HAC' uses Newey-West method